In [16]:
from pynq import PL
PL.reset() #important fixes caching issues which have popped up.
import xrfdc #poorly documented library that handles interfacing to the RF data converter
from pynq import Overlay  #import the overlay module
ol = Overlay('./design_1_wrapper.bit')  #locate/point to the bit file
import pprint
pprint.pprint(ol.ip_dict)
dma = ol.axi_dma_0 #might need to change name depending on what you called it
rf = ol.usp_rf_data_converter_0 #might need to change name depending on what you called it


{'axi_dma_0': {'addr_range': 65536,
               'bdtype': None,
               'device': <pynq.pl_server.embedded_device.EmbeddedDevice object at 0xffffa6da6a40>,
               'driver': <class 'pynq.lib.dma.DMA'>,
               'fullpath': 'axi_dma_0',
               'gpio': {},
               'interrupts': {},
               'mem_id': 'S_AXI_LITE',
               'memtype': 'REGISTER',
               'parameters': {'ADDR_WIDTH': '10',
                              'ARUSER_WIDTH': '0',
                              'AWUSER_WIDTH': '0',
                              'BUSER_WIDTH': '0',
                              'CLK_DOMAIN': 'design_1_zynq_ultra_ps_e_0_0_pl_clk0',
                              'C_BASEADDR': '0xA0000000',
                              'C_DLYTMR_RESOLUTION': '125',
                              'C_ENABLE_MULTI_CHANNEL': '0',
                              'C_FAMILY': 'zynquplus',
                              'C_HIGHADDR': '0xA000FFFF',
                          

In [17]:
adc_tile = rf.adc_tiles[2]
adc_block = adc_tile.blocks[0]
print(adc_block)
print(adc_block.BlockStatus)
print(adc_block.MixerSettings)
adc_block.Dither = 0 #doesn't really matter for this lab, but let's turn off.
adc_block.MixerSettings['Freq']= 0 # set the frequency of the Numerically controlled oscillator.
adc_block.UpdateEvent(xrfdc.EVENT_MIXER) #every time setting is changed, must call this.
print(adc_block.MixerSettings)

{'SamplingFreq': 1.47456, 'AnalogDataPathStatus': 1, 'DigitalDataPathStatus': 929, 'DataPathClocksStatus': 1, 'IsFIFOFlagsEnabled': 3, 'IsFIFOFlagsAsserted': 0}
{'Freq': 0.0, 'PhaseOffset': 0.0, 'EventSource': 2, 'CoarseMixFreq': 16, 'MixerMode': 3, 'FineMixerScale': 0, 'MixerType': 2}
{'Freq': 0.0, 'PhaseOffset': 0.0, 'EventSource': 2, 'CoarseMixFreq': 0, 'MixerMode': 3, 'FineMixerScale': 0, 'MixerType': 2}


In [18]:
adc_block.MixerSettings['Freq']= 74.25
adc_block.UpdateEvent(xrfdc.EVENT_MIXER) #every time setting is changed, must call this.
print(adc_block.MixerSettings)

{'Freq': 74.24999999999477, 'PhaseOffset': 0.0, 'EventSource': 2, 'CoarseMixFreq': 0, 'MixerMode': 3, 'FineMixerScale': 0, 'MixerType': 2}


In [19]:
import numpy as np
import time
from pynq import allocate
import math
out_buffer = allocate(400024, dtype=np.int64)
old = 0
duration = 0.05
first = 0
threshold = 4830648 / 2


start_time = time.time()

bit_stream = []
x = 0
while True:
    while (x < threshold):
        dma.recvchannel.transfer(out_buffer)
        dma.recvchannel.wait()
        x = out_buffer[0]
    i = 0
    start_time = time.time()
    while (i < 50):
        while (x > threshold):
            dma.recvchannel.transfer(out_buffer)
            dma.recvchannel.wait()
            x = out_buffer[0]
        stop_time = time.time()
        print("1" * round((stop_time - start_time)/duration), end="")
        i += round((stop_time - start_time)/duration)
        start_time = time.time()
        while (x < threshold):
            dma.recvchannel.transfer(out_buffer)
            dma.recvchannel.wait()
            x = out_buffer[0]
        stop_time = time.time()
        print("0" * round((stop_time - start_time)/duration), end="")
        i += round((stop_time - start_time)/duration)
        start_time = time.time()



000000000000000000100101100010011000000110001001100111110000000000011111

KeyboardInterrupt: 

In [20]:
s = "00101100010011000000110001001100"

def binary_to_ascii(binary_string):
    # Split the binary string into groups of 8 bits
    ascii_characters = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]
    # Convert each 8-bit binary group to an ASCII character
    ascii_text = ''.join([chr(int(char, 2)) for char in ascii_characters])
    return ascii_text

# Example usage
ascii_output = binary_to_ascii(s[::-1])
print(ascii_output)  # Outputs: HELLO

2024
